# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [598]:
problem = '''Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals: 
You have a target of selling 5500 units. 
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target. 

Employee Hours: 
You have both full-time and part-time employees. 
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours. 
Similarly, like sales, there could be deviations in these working hours. 

Overtime: 
Additionally, your employees might end up working overtime. 
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you. 
P1 to P4 are priority factors for each deviation. 
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities. 
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business.'''

## 2. Generate the mathematical model

In [618]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [619]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Employee Management

**Sets:**

* E = {Full-Time, Part-Time} : Set of employee types

**Parameters:**

* TargetSales = 5500 units : Target sales
* TargetHours_FullTime = 800 hours : Target working hours for full-time employees
* TargetHours_PartTime = 320 hours : Target working hours for part-time employees
* TargetOvertime_FullTime = 100 hours : Target overtime hours for full-time employees
* P1 : Priority factor for sales deviation
* P2 : Priority factor for full-time employee working hours deviation
* P3 : Priority factor for part-time employee working hours deviation
* P4 : Priority factor for full-time employee overtime deviation

**Decision Variables:**

* Sales : Actual sales achieved
* Hours_FullTime : Actual working hours of full-time employees
* Hours_PartTime : Actual working hours of part-time employees
* Overtime_FullTime : Actual overtime hours of full-time employees
* Overtime_PartTime : Actual overtime hours of part-time employees

**Objective Function:**

Minimize the weighted sum of deviations from the targets, considering their priorities:

```
Minimize:
  P1 * | Sales - TargetSales | +
  2 * P2 * | Hours_FullTime - TargetHours_FullTime | +
  P3 * | Hours_PartTime - TargetHours_PartTime | +
  P4 * | Overtime_FullTime - TargetOvertime_FullTime | +
  3 * P4 * Overtime_PartTime 
```

**Constraints:**

None explicitly stated in the problem description. However, you might want to add constraints based on practical limitations, such as:

*  Non-negativity constraints for all decision variables:
    *  Sales >= 0
    *  Hours_FullTime >= 0
    *  Hours_PartTime >= 0
    *  Overtime_FullTime >= 0
    *  Overtime_PartTime >= 0

*  Maximum working hours for each employee type based on legal or contractual agreements.

This model provides a basic framework for optimizing employee management in your shop. You can adapt it by incorporating additional constraints and parameters relevant to your specific context. Remember to choose appropriate values for the priority factors (P1-P4) to reflect your business priorities. 


## 3. Generate the pyomo code

In [620]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [621]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Sets
model.E = pyo.Set(initialize=['Full-Time', 'Part-Time'])

# Parameters (Sample Data)
model.TargetSales = 5500
model.TargetHours = {'Full-Time': 800, 'Part-Time': 320}
model.TargetOvertime = {'Full-Time': 100, 'Part-Time': 0}  # Assuming no target overtime for part-time
model.P1 = 1  # Priority for sales deviation (Sample Data)
model.P2 = 0.8  # Priority for full-time working hours deviation (Sample Data)
model.P3 = 0.5  # Priority for part-time working hours deviation (Sample Data)
model.P4 = 1.2  # Priority for full-time overtime deviation (Sample Data)

# Decision Variables
model.Sales = pyo.Var(within=pyo.NonNegativeReals)
model.Hours = pyo.Var(model.E, within=pyo.NonNegativeReals)
model.Overtime = pyo.Var(model.E, within=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    return (
        model.P1 * abs(model.Sales - model.TargetSales)
        + 2 * model.P2 * abs(model.Hours['Full-Time'] - model.TargetHours['Full-Time'])
        + model.P3 * abs(model.Hours['Part-Time'] - model.TargetHours['Part-Time'])
        + model.P4 * abs(model.Overtime['Full-Time'] - model.TargetOvertime['Full-Time'])
        + 3 * model.P4 * model.Overtime['Part-Time']
    )
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints (You can add more constraints as needed)
# Non-negativity constraints are already defined in the decision variables

# Solve the model
solver = pyo.SolverFactory('cbc')  # You can choose a different solver if needed
solver.solve(model)

# Print results
print("Sales:", pyo.value(model.Sales))
print("Full-Time Hours:", pyo.value(model.Hours['Full-Time']))
print("Part-Time Hours:", pyo.value(model.Hours['Part-Time']))
print("Full-Time Overtime:", pyo.value(model.Overtime['Full-Time']))
print("Part-Time Overtime:", pyo.value(model.Overtime['Part-Time']))
```

**Sample Data Used:**

* Priority factors (P1-P4) are given sample values. You should adjust these according to your business priorities. 
* TargetOvertime for 'Part-Time' is assumed to be 0.

Remember to install the necessary packages (pyomo and a solver like cbc) before running this code. 


## 4. Input problem data and try running the generated code

In [622]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Sets
model.E = pyo.Set(initialize=['Full-Time', 'Part-Time'])

# Parameters (Sample Data)
model.TargetSales = 5500
model.TargetHours = {'Full-Time': 800, 'Part-Time': 320}
model.TargetOvertime = {'Full-Time': 100, 'Part-Time': 0}  # Assuming no target overtime for part-time
model.P1 = 1  # Priority for sales deviation (Sample Data)
model.P2 = 0.8  # Priority for full-time working hours deviation (Sample Data)
model.P3 = 0.5  # Priority for part-time working hours deviation (Sample Data)
model.P4 = 1.2  # Priority for full-time overtime deviation (Sample Data)

# Decision Variables
model.Sales = pyo.Var(within=pyo.NonNegativeReals)
model.Hours = pyo.Var(model.E, within=pyo.NonNegativeReals)
model.Overtime = pyo.Var(model.E, within=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    return (
        model.P1 * abs(model.Sales - model.TargetSales)
        + 2 * model.P2 * abs(model.Hours['Full-Time'] - model.TargetHours['Full-Time'])
        + model.P3 * abs(model.Hours['Part-Time'] - model.TargetHours['Part-Time'])
        + model.P4 * abs(model.Overtime['Full-Time'] - model.TargetOvertime['Full-Time'])
        + 3 * model.P4 * model.Overtime['Part-Time']
    )
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints (You can add more constraints as needed)
# Non-negativity constraints are already defined in the decision variables

# Solve the model
solver = pyo.SolverFactory('ipopt')  # You can choose a different solver if needed
solver.solve(model)

# Print results
print("Sales:", pyo.value(model.Sales))
print("Full-Time Hours:", pyo.value(model.Hours['Full-Time']))
print("Part-Time Hours:", pyo.value(model.Hours['Part-Time']))
print("Full-Time Overtime:", pyo.value(model.Overtime['Full-Time']))
print("Part-Time Overtime:", pyo.value(model.Overtime['Part-Time']))

model.name="unknown";
    - termination condition: maxIterations
    - message from solver: Ipopt 3.11.1\x3a Maximum Number of Iterations
      Exceeded.
Sales: 5500.003213097577
Full-Time Hours: 1201.311957548214
Part-Time Hours: 445.68569192693786
Full-Time Overtime: 101.92481008697638
Part-Time Overtime: 0.027777709021643098


## 5. Correct the code to verify model viability (optional)

In [623]:
model.obj()

707.3549629950378

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [624]:
print(response.text)

## Mathematical Optimization Model for Employee Management

**Sets:**

* E = {Full-Time, Part-Time} : Set of employee types

**Parameters:**

* TargetSales = 5500 units : Target sales
* TargetHours_FullTime = 800 hours : Target working hours for full-time employees
* TargetHours_PartTime = 320 hours : Target working hours for part-time employees
* TargetOvertime_FullTime = 100 hours : Target overtime hours for full-time employees
* P1 : Priority factor for sales deviation
* P2 : Priority factor for full-time employee working hours deviation
* P3 : Priority factor for part-time employee working hours deviation
* P4 : Priority factor for full-time employee overtime deviation

**Decision Variables:**

* Sales : Actual sales achieved
* Hours_FullTime : Actual working hours of full-time employees
* Hours_PartTime : Actual working hours of part-time employees
* Overtime_FullTime : Actual overtime hours of full-time employees
* Overtime_PartTime : Actual overtime hours of part-time employees


In [625]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Sets
model.E = pyo.Set(initialize=['Full-Time', 'Part-Time'])

# Parameters (Sample Data)
model.TargetSales = 5500
model.TargetHours = {'Full-Time': 800, 'Part-Time': 320}
model.TargetOvertime = {'Full-Time': 100, 'Part-Time': 0}  # Assuming no target overtime for part-time
model.P1 = 1  # Priority for sales deviation (Sample Data)
model.P2 = 0.8  # Priority for full-time working hours deviation (Sample Data)
model.P3 = 0.5  # Priority for part-time working hours deviation (Sample Data)
model.P4 = 1.2  # Priority for full-time overtime deviation (Sample Data)

# Decision Variables
model.Sales = pyo.Var(within=pyo.NonNegativeReals)
model.Hours = pyo.Var(model.E, within=pyo.NonNegativeReals)
model.Overtime = pyo.Var(model.E, within=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    return (
        model.P1 * abs(model.Sales - model.TargetSales)
        + 2 * model.P2 * abs(mo